In [107]:
import numpy    as np
from numpy.testing._private.utils import decorate_methods
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl
import time

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

from sklearn.tree          import DecisionTreeRegressor
from sklearn.ensemble      import RandomForestRegressor
from sklearn.ensemble      import ExtraTreesRegressor
from sklearn.ensemble      import AdaBoostRegressor
from sklearn.ensemble      import GradientBoostingRegressor
from xgboost               import XGBRegressor
from lightgbm              import LGBMRegressor
from catboost              import CatBoostRegressor

In [108]:
""" Read CSV file """

data = pd.read_csv(r'data\london_merged.csv')

data.head(4)

,timestamp,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0
1,2015-01-04 01:00:00,138,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0
2,2015-01-04 02:00:00,134,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0
3,2015-01-04 03:00:00,72,2.0,2.0,100.0,0.0,1.0,0.0,1.0,3.0


In [109]:
""" Set the seed. This will make the np.random predictable """
np.random.seed(0)

In [110]:
""" Split the data into x and y  """

# target = data['cnt']
# data = data.drop(['cnt'], axis=1)

# data.head(3)
#target.head(3)

' Split the data into x and y  '

In [111]:
# # Print data shape
# print(target.shape)
# print(data.shape)

# # Take a look at nulls 0 nulls
# print(target.isnull().sum())
# print(data.isnull().sum())

In [112]:
data['year'] = data['timestamp'].apply(lambda row: row[:4])
data['month'] = data['timestamp'].apply(lambda row: row.split('-')[2][:2] )
data['hour'] = data['timestamp'].apply(lambda row: row.split(':')[0][-2:] )

data.drop('timestamp', axis=1, inplace=True)

data.head(3)

,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season,year,month,hour
0,182,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0,2015,04,00
1,138,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0,2015,04,01
2,134,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0,2015,04,02


In [113]:
def data_enhancement(data):
    
    gen_data = data
    
    for season in data['season'].unique():
        seasonal_data =  gen_data[gen_data['season'] == season]
        hum_std = seasonal_data['hum'].std()
        wind_speed_std = seasonal_data['wind_speed'].std()
        t1_std = seasonal_data['t1'].std()  
        t2_std = seasonal_data['t2'].std() 
        
        for i in gen_data[gen_data['season'] == season].index:
            if np.random.randint(2) == 1:
                gen_data['hum'].values[i] += hum_std/10
            else:
                gen_data['hum'].values[i] -= hum_std/10
                
            if np.random.randint(2) == 1:
                gen_data['wind_speed'].values[i] += wind_speed_std/10
            else:
                gen_data['wind_speed'].values[i] -= wind_speed_std/10
                
            if np.random.randint(2) == 1:
                gen_data['t1'].values[i] += t1_std/10
            else:
                gen_data['t1'].values[i] -= t1_std/10
                
            if np.random.randint(2) == 1:
                gen_data['t2'].values[i] += t2_std/10
            else:
                gen_data['t2'].values[i] -= t2_std/10

    return gen_data

In [114]:
print(data.head(3))

gen = data_enhancement(data)
print(gen.head(3) )

#print(gen.shape)

   cnt   t1   t2   hum  wind_speed  weather_code  is_holiday  is_weekend  \
0  182  3.0  2.0  93.0         6.0           3.0         0.0         1.0   
1  138  3.0  2.5  93.0         5.0           1.0         0.0         1.0   
2  134  2.5  2.5  96.5         0.0           1.0         0.0         1.0   

   season  year month hour  
0     3.0  2015    04   00  
1     3.0  2015    04   01  
2     3.0  2015    04   02  
   cnt        t1       t2        hum  wind_speed  weather_code  is_holiday  \
0  182  3.379372  1.51169  91.910483    6.890895           3.0         0.0   
1  138  3.379372  2.98831  94.089517    5.890895           1.0         0.0   
2  134  2.879372  2.01169  97.589517    0.890895           1.0         0.0   

   is_weekend  season  year month hour  
0         1.0     3.0  2015    04   00  
1         1.0     3.0  2015    04   01  
2         1.0     3.0  2015    04   02  


In [115]:
#final_data = data
y = data['cnt']
x = data.drop(['cnt'], axis=1)

In [116]:
cat_vars = ['season','is_weekend','is_holiday','year','month','weather_code']
num_vars = ['t1','t2','hum','wind_speed']

x_train, x_val, y_train, y_val = model_selection.train_test_split(x, y, test_size=0.2, random_state=0 ) # Recommended for reproducibility

In [117]:
extra_sample = gen.sample(gen.shape[0] // 3)

x_train = pd.concat([x_train, extra_sample.drop(['cnt'], axis=1 ) ])
y_train = pd.concat([y_train, extra_sample['cnt'] ])


transformer = preprocessing.PowerTransformer()

y_train = transformer.fit_transform(y_train.values.reshape(-1,1))
y_val = transformer.transform(y_val.values.reshape(-1,1))

In [118]:

rang = abs(y_train.max()) + abs(y_train.min())

In [127]:
num_4_treeModels = pipeline.Pipeline(steps=[  ('imputer', impute.SimpleImputer(strategy='constant', fill_value=-9999)),])

cat_4_treeModels = pipeline.Pipeline(steps=[ ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')),  ('ordinal', preprocessing.OrdinalEncoder()) ])

tree_prepro = compose.ColumnTransformer(transformers=[     ('num', num_4_treeModels, num_vars),  ('cat', cat_4_treeModels, cat_vars),  ], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

In [128]:
tree_classifiers = {
  "Decision Tree": DecisionTreeRegressor(),
  "Extra Trees":   ExtraTreesRegressor(n_estimators=100),
  "Random Forest": RandomForestRegressor(n_estimators=100),
  "AdaBoost":      AdaBoostRegressor(n_estimators=100),
  "Skl GBM":       GradientBoostingRegressor(n_estimators=100),
  "XGBoost":       XGBRegressor(n_estimators=100),
  "LightGBM":      LGBMRegressor(n_estimators=100),
  "CatBoost":      CatBoostRegressor(n_estimators=100),
}
### END SOLUTION


In [129]:
tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}

results = pd.DataFrame({'Model': [], 'MSE': [], 'MAB': [], " % error": [], 'Time': []})

In [130]:
for model_name, model in tree_classifiers.items():
    
    start_time = time.time()
    model.fit(x_train, y_train)
    total_time = time.time() - start_time
        
    pred = model.predict(x_val)
    
    results = results.append({"Model":    model_name,
                              "MSE": metrics.mean_squared_error(y_val, pred),
                              "MAB": metrics.mean_absolute_error(y_val, pred),
                              " % error": metrics.mean_squared_error(y_val, pred) / rang,
                              "Time":     total_time},
                              ignore_index=True)
### END SOLUTION

C:\Users\Omistaja\.conda\envs\pz\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\Omistaja\.conda\envs\pz\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\Omistaja\.conda\envs\pz\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Omistaja\.conda\envs\pz\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array

Learning rate set to 0.42641
0:	learn: 0.8888793	total: 12.5ms	remaining: 1.24s
1:	learn: 0.8442251	total: 19.4ms	remaining: 951ms
2:	learn: 0.8215277	total: 23.4ms	remaining: 755ms
3:	learn: 0.8078977	total: 26.8ms	remaining: 643ms
4:	learn: 0.7997043	total: 29.9ms	remaining: 568ms
5:	learn: 0.7936340	total: 33.3ms	remaining: 521ms
6:	learn: 0.7888441	total: 37.8ms	remaining: 502ms
7:	learn: 0.7868824	total: 41.9ms	remaining: 482ms
8:	learn: 0.7817758	total: 44.8ms	remaining: 453ms
9:	learn: 0.7801680	total: 47.8ms	remaining: 430ms
10:	learn: 0.7778249	total: 50.9ms	remaining: 412ms
11:	learn: 0.7757535	total: 54ms	remaining: 396ms
12:	learn: 0.7743679	total: 56.7ms	remaining: 380ms
13:	learn: 0.7720303	total: 59.5ms	remaining: 366ms
14:	learn: 0.7695139	total: 63ms	remaining: 357ms
15:	learn: 0.7686639	total: 67.1ms	remaining: 352ms
16:	learn: 0.7672573	total: 70.4ms	remaining: 344ms
17:	learn: 0.7665251	total: 73.4ms	remaining: 334ms
18:	learn: 0.7647027	total: 76.3ms	remaining: 325

In [132]:

results_ord = results.sort_values(by=['MSE'], ascending=True, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['MSE', 'MAE'], vmin=0, vmax=100, color='#5fba7d')

print(results_ord)

           Model       MSE       MAB   % error      Time
1    Extra Trees  0.388857  0.373341  0.070435  3.988633
2  Random Forest  0.422930  0.452506  0.076607  6.397737
3        XGBoost  0.532979  0.554602  0.096540  0.649979
4       LightGBM  0.563719  0.577875  0.102109  0.238394
5       CatBoost  0.581709  0.584727  0.105367  2.643735
6        Skl GBM  0.625755  0.614863  0.113345  1.999980
7       AdaBoost  0.694637  0.664619  0.125822  0.403651
8  Decision Tree  0.758544  0.523285  0.137398  0.204242


In [133]:
print(y_train.max())
print(y_train.min())
print(y_val[3])
print(tree_classifiers['Random Forest'].predict(x_val)[3])

2.766201853700867
-2.754580496720054
[0.56668132]
1.0555838689004904
